In [25]:
#para leer el archivo PDF desde la ruta del repositorio.
!pip install PyPDF2
#para realizar el análisis de diseño y extraer texto y formato del PDF. (la versión .six de la biblioteca es la que admite Python 3)
!pip install pdfminer.six
#para identificar tablas en una página PDF y extraer la información de ellas.
!pip install pdfplumber
#para convertir la imagen PDF recortada a una imagen PNG.
!pip install pdf2image
#Para leer la imagen PNG.
!pip install Pillow
#Poppler es una biblioteca de software que se utiliza para trabajar con archivos PDF.
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [8]:
#para extraer el texto de las imágenes utilizando tecnología OCR.
!pip install pytesseract
!apt-get install tesseract-ocr
#si el entonrno es en windows, primero debe instalar Google Tesseract OCR , que es una máquina de OCR basada en un modelo LSTM para identificar el reconocimiento de líneas y patrones de caracteres.
#link ==> https://linuxhint.com/install-tesseract-windows/
#pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# To perform OCR to extract text from images
import pytesseract
print(pytesseract.get_tesseract_version())


In [28]:
# To read the PDF
import PyPDF2
# To analyze the PDF layout and extract text
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# To extract text from tables in PDF
import pdfplumber
# To extract the images from the PDFs
from PIL import Image
from pdf2image import convert_from_path
# To remove the additional created files
import os


Para el análisis preliminar, utilizamos la biblioteca PDFMiner Python para separar el texto de un objeto de documento en múltiples objetos de página y luego desglosar y examinar el diseño de cada página. Los archivos PDF carecen inherentemente de información estructurada, como párrafos, oraciones o palabras vistas por el ojo humano. En cambio, sólo entienden los caracteres individuales del texto junto con su posición en la página.

In [12]:
pdf_path = "/content/Promt Paterns.pdf"
pdf_path

'/content/Promt Paterns.pdf'

In [18]:
for pagenum, page in enumerate(extract_pages(pdf_path)):

    # Iterar los elementos que componen una página
    for element in page:

        # Comprobar si el elemento es un elemento de texto
        if isinstance(element, LTTextContainer):
            # Function to extract text from the text block
            pass
            # Función para extraer formato de texto
            pass

        # Verifique los elementos para imágenes
        if isinstance(element, LTFigure):
            # Función para convertir PDF a imagen
            pass
            # Función para extraer texto con OCR
            pass

        # Check the elements for tables
        if isinstance(element, LTRect):
            # Función para extraer la tabla
            pass
            # Función para convertir el contenido de la tabla en un string
            pass

#Definir la función para extraer texto de PDF
Entonces, para extraer texto de un contenedor de texto, simplemente usamos el método get_text () del elemento LTTextContainer. Este método recupera todos los caracteres que componen las palabras dentro del cuadro de corpus específico, almacenando el resultado en una lista de datos de texto. Cada elemento de esta lista representa la información textual sin procesar contenida en el contenedor.

Ahora, para identificar el formato de este texto, iteramos a través del objeto LTTextContainer para acceder a cada línea de texto de este corpus individualmente. En cada iteración, se crea un nuevo objeto LTTextLine , que representa una línea de texto en este fragmento de corpus. Luego examinamos si el elemento de línea anidado contiene texto. Si es así, accedemos a cada elemento de carácter individual como LTChar, que contiene todos los metadatos de ese carácter. De estos metadatos, extraemos dos tipos de formatos y los almacenamos en una lista separada, ubicada correspondientemente al texto examinado:

La familia de fuentes de los caracteres, incluido si el carácter está en formato negrita o cursiva.
El tamaño de fuente del carácter.
Generalmente, los caracteres dentro de un fragmento específico de texto tienden a tener un formato consistente a menos que algunos estén resaltados en negrita. Para facilitar un análisis más detallado, capturamos los valores únicos de formato de texto para todos los caracteres del texto y los almacenamos en la lista adecuada.



In [17]:
# Crear una función para extraer texto

def text_extraction(element):
    # Extraer el texto del elemento de texto en línea
    line_text = element.get_text()

    # Encontrar los formatos del texto
    # Inicializar la lista con todos los formatos que aparecieron en el línea de texto
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Iterando a través de cada carácter en la línea de texto
            for character in text_line:
                if isinstance(character, LTChar):
                    # Agregue el nombre de fuente del personaje
                    line_formats.append(character.fontname)
                    # Agrega el tamaño de fuente del carácter
                    line_formats.append(character.size)
    # Encuentra los tamaños de fuente únicos y los nombres en la línea
    format_per_line = list(set(line_formats))

    # Devuelve una tupla con el texto en cada línea junto con su formato
    return (line_text, format_per_line)

# Definir la función para extraer texto de imágenes
En primer lugar, debemos establecer aquí que los elementos de imagen almacenados en archivos PDF no estén en un formato diferente al del archivo, como JPEG o PNG. De esa manera, para aplicarles el software OCR, primero debemos separarlos del archivo y luego convertirlos a un formato de imagen.

Para lograrlo seguimos el siguiente proceso:

Usamos los metadatos del objeto LTFigure detectado por PDFMiner para recortar el cuadro de imagen, utilizando sus coordenadas en el diseño de la página. Luego lo guardamos como un nuevo PDF en nuestro directorio usando la biblioteca PyPDF2 .
Luego empleamos la función convert_from_file () de la biblioteca pdf2image para convertir todos los archivos PDF en el directorio en una lista de imágenes, guardándolas en formato PNG.
Finalmente, ahora que tenemos nuestros archivos de imagen, los leemos en nuestro script usando el paquete Imagen del módulo PIL e implementamos la función image_to_string () de pytesseract para extraer texto de las imágenes usando el motor OCR tesseract.
Como resultado, este proceso devuelve el texto de las imágenes, que luego guardamos en una tercera lista dentro del diccionario de salida. Esta lista contiene la información textual extraída de las imágenes de la página examinada.

In [16]:
# Crear una función para recortar los elementos de la imagen de los archivos PDF
def crop_image(element, pageObj):
    # Obtener las coordenadas para recortar la imagen del PDF
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1]
    # Recorta la página usando coordenadas (izquierda, abajo, derecha, arriba)
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    # Guarda la página recortada en un nuevo PDF
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    # Guarde el PDF recortado en un nuevo archivo
    with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

# Crea una función para convertir el PDF a imágenes
def convert_to_images(input_file,):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = "PDF_image.png"
    image.save(output_file, "PNG")

# Crea una función para leer texto de imágenes
def image_to_text(image_path):
    # Lee la imagen
    img = Image.open(image_path)
    # Extrae el texto de la imagen
    text = pytesseract.image_to_string(img)
    return text

#Definir la función para extraer texto de Tablas

Para abordar esta tarea utilizamos la biblioteca pdfplomber por varios motivos. En primer lugar, está construido sobre pdfminer.six que utilizamos para nuestro análisis preliminar, lo que significa que contiene objetos similares. Además, su enfoque para la detección de tablas se basa en elementos de línea junto con sus intersecciones que construyen la celda que contiene el texto y luego la tabla misma. De esa manera, después de identificar una celda de una tabla, podemos extraer solo el contenido dentro de la celda sin tener que saber cuántas filas deben representarse. Luego, cuando tengamos el contenido de una tabla, le daremos formato en una cadena similar a una tabla y la almacenaremos en la lista apropiada.

Para lograr eso, creamos dos funciones, extract_table() para extraer el contenido de la tabla en una lista de listas, y table_converter() para unir el contenido de esas listas en una cadena similar a una tabla.

En la función extract_table() :

Abrimos el archivo PDF.
Navegamos hasta la página examinada del archivo PDF.
De la lista de tablas que se encuentra en la página de pdfplomber, seleccionamos la deseada.
Extraemos el contenido de la tabla y lo generamos en una lista de listas anidadas que representan cada fila de la tabla.

En la función table_converter() :

Iteramos en cada lista anidada y limpiamos su contexto de cualquier salto de línea no deseado proveniente de cualquier texto ajustado.
Unimos cada elemento de la fila separándolos mediante la tecla | símbolo para crear la estructura de la celda de una tabla.
Finalmente, agregamos un salto de línea al final para pasar a la siguiente fila.
Esto dará como resultado una cadena de texto que presentará el contenido de la tabla sin perder la granularidad de los datos presentados en ella

In [19]:
# Extraer tablas de la página

def extract_table(pdf_path, page_num, table_num):
    # Abrir el archivo pdf
    pdf = pdfplumber.open(pdf_path)
    # Encuentra la página examinada
    table_page = pdf.pages[page_num]
    # Extrae la tabla apropiada
    table = table_page.extract_tables()[table_num]
    return table

# Convierte la tabla al formato apropiado
def table_converter(table):
    table_string = ''
    # Iterar a través de cada fila de la tabla
    for row_num in range(len(table)):
        row = table[row_num]
        # Elimina el salto de línea de los textos ajustados.
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Convertimos la tabla a string
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Eliminamos el último salto de línea
    table_string = table_string[:-1]
    return table_string

#Unimos todo junto.

Importe las bibliotecas necesarias.

Abra el archivo PDF usando la biblioteca pyPDF2 .

Extraiga cada página del PDF y repita los siguientes pasos.

Examine si hay tablas en la página y cree una lista de ellas usando pdfplumner.

Encuentre todos los elementos anidados en la página y ordénelos tal como aparecieron en su diseño.

Luego para cada elemento:

Examina si es un contenedor de texto y no aparece en un elemento de tabla. Luego use la función text_extraction () para extraer el texto junto con su formato; de lo contrario, pase este texto.

Examine si es una imagen y use la función crop_image () para recortar el componente de imagen del PDF, conviértalo en un archivo de imagen usando convert_to_images ( ) y extraiga texto usando OCR con la función image_to_text ().

Examina si es un elemento rectangular. En este caso, examinamos si el primer rect es parte de la tabla de una página y, en caso afirmativo, pasamos a los siguientes pasos:

Busque el cuadro delimitador de la tabla para no volver a extraer su texto con la función text_extraction().
Extrae el contenido de la tabla y conviértelo en una cadena.
Luego agregue un parámetro booleano para aclarar que extraemos texto de la tabla.
Este proceso finalizará después del último LTRect que cae en el cuadro delimitador de la tabla y el siguiente elemento en el diseño no es un objeto rectangular. (Se pasarán todos los demás objetos que componen la tabla)
Las salidas del proceso se almacenarán en 5 listas por iteración, denominadas:

page_text: contiene el texto proveniente de contenedores de texto en el PDF (el marcador de posición se colocará cuando el texto se extraiga de otro elemento)
line_format: contiene los formatos de los textos extraídos arriba (el marcador de posición se colocará cuando el texto se extraiga de otro elemento)
text_from_images: contiene los textos extraídos de las imágenes de la página
text_from_tables: contiene la cadena similar a una tabla con el contenido de las tablas
page_content: contiene todo el texto representado en la página en una lista de elementos
Todas las listas se almacenarán bajo la clave de un diccionario que representará el número de página examinada cada vez.

Posteriormente cerraremos el archivo PDF.

Luego eliminaremos todos los archivos adicionales creados durante el proceso.

Por último, podemos mostrar el contenido de la página uniendo los elementos de la lista page_content.



4.1.1


In [36]:
# Encuentra la ruta del PDF
pdf_path = "impacto-regulatorio-mica-en los-criptoactivos-pwc.pdf"

# crea un objeto de archivo PDF
pdfFileObj = open(pdf_path, 'rb')
# crea un objeto de lector de PDF
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Crea el diccionario para extraer texto de cada imagen
text_per_page = {}
# Extraemos las páginas del PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):

    # Inicializamos las variables necesarias para la extracción de texto de la página
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Inicializa el número de tablas examinadas
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Abre el archivo pdf
    pdf = pdfplumber.open(pdf_path)
    # Encuentra la página examinada
    page_tables = pdf.pages[pagenum]
    # Encuentra el número de tablas en la página
    tables = page_tables.find_tables()


    # Encuentra todos los elementos
    page_elements = [(element.y1, element) for element in page._objs]
    # Ordena todos los elementos tal como aparecen en la página
    page_elements.sort(key=lambda a: a[0], reverse=True)

    # Encuentra los elementos que componen una página
    for i,component in enumerate(page_elements):
        # Extrae la posición de la parte superior del elemento en el PDF
        pos= component[0]
        # Extrae el elemento de diseño de página
        element = component[1]

        # Verifique si el elemento es un elemento de texto
        if isinstance(element, LTTextContainer):
            # Verifique si el texto apareció en una tabla
            if table_extraction_flag == False:
                # Use la función para extraer el texto y formato para cada elemento de texto
                (line_text, format_per_line) = text_extraction(element)
                # Agregar el texto de cada línea al texto de la página
                page_text.append(line_text)
                # Agregar el formato para cada línea que contiene texto
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Omitir el texto que apareció en una tabla
                pass

        # Verificar los elementos en busca de imágenes
        if isinstance(element, LTFigure):
            # Recortar la imagen del PDF
            crop_image(element, pageObj)
            # Convertir el pdf recortado en una imagen
            convert_to_images('cropped_image.pdf')
            # Extraer el texto de la imagen
            image_text = image_to_text('PDF_image.png')
            text_from_images.append(image_text)
            page_content.append(image_text)
            # Agregar un marcador de posición en las listas de texto y formato
            page_text.append('image')
            line_format.append('image')

        # Verifique los elementos de las tablas
        if isinstance(element, LTRect):
            # Si el primer elemento rectangular
            if first_element == True and (table_num+1) <= len(tables):
                # Encuentre el cuadro delimitador de la tabla
                lower_side = page.bbox[3] - tables[table_num].bbox[3]
                upper_side = element.y1
                # Extraer la información de la tabla
                table = extract_table(pdf_path, pagenum, table_num)
                # Convertir la información de la tabla en formato de cadena estructurada
                table_string = table_converter(table)
                # Agrega la cadena de la tabla a una lista
                text_from_tables.append(table_string)
                page_content.append(table_string)
                # Establece como True para evitar el contenido nuevamente
                table_extraction_flag = True
                # Conviértelo en otro elemento
                first_element = False
                # Agregar un marcador de posición en las listas de texto y formato
                page_text.append('table')
                line_format.append('table')

            # Comprobar si ya extrajimos las tablas de la página
            if element.y0 >= lower_side and element.y1 <= upper_side:
                pass
            elif not isinstance(page_elements[i+1][1], LTRect):
                table_extraction_flag = False
                first_element = True
                table_num+=1


    # Crear la clave del diccionario
    dctkey = 'Page_'+str(pagenum)
    # Agregar la lista de listas como el valor de la clave de página
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

# Cerrar el objeto del archivo pdf
pdfFileObj.close()

# Eliminar los archivos adicionales creados
os.remove('cropped_image.pdf')
os.remove('PDF_image.png')

# Mostrar el contenido de la página
result = ''.join(text_per_page['Page_0'][4])
print(result)


|El impacto regulatorio de la Propuesta MiCA Proposal for a Regulation of the European Parliament and of the Council on Markets in Crypto-assets, and amending Directive (EU) 2019/1937COM/2020/593||
||None| 
 

pwe



In [40]:
result = ''.join(text_per_page['Page_4'][4])
print(result)

 
El impacto regulatorio de la 
Propuesta MiCA 
1.  Contexto  
 
Durante la última década se ha configurado el sistema financiero como uno de los 
sectores más regulados, especialmente después de la crisis financiera mundial de 
2008. En los años siguientes, los esfuerzos coordinados de los organismos 
internacionales supervisores han avanzado en la preservación de los dos objetivos 
principales: el mantenimiento de la estabilidad financiera y la protección del 
consumidor. Ambos constituyen el punto de partida de las regulaciones financieras, y 
estructuran el marco normativo aplicable al ecosistema de los criptoactivos. 
Finance Package, que incluye, entre 
otros, una propuesta de regulación del 
mercado de criptoactivos, conocida como 
propuesta MiCA (siglas en inglés de 
Market in Crypto Assets)1.  
El fenómeno de las criptomonedas nace 
en el año 2008 con la publicación del 
paper de Bitcoin. Mas recientemente, el 
surgimiento de iniciativas privadas con 
potencial sistémico, como